Some initial comments: Even as an EDA assignment, this is a work in progress. I've played around quite a bit with the data, and I think some of that is a reflection of my level of comfort (or lack thereof) with some modeling processes. Still, I hope this constitutes a reasonable beginning. 

In [1]:
import matplotlib.pyplot as plt

% matplotlib inline

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import datetime as DT
import collections, os


os.chdir('C:\Users\jhoukal\Desktop\New folder')

files = [f for f in os.listdir(os.getcwd()) if f.endswith('csv')]
files #I'd like to make this part dynamic, but for the meantime I at least get a quick list of the files I want to upload

['bigfile.csv',
 'countmerge.csv',
 'EventData.csv',
 'gatewayEvents.csv',
 'gw2.csv',
 'gwold.csv',
 'LoginData.csv',
 'PartnerData.csv',
 'pwresets.csv',
 'pw_data.csv',
 'Salesforce.csv',
 'saveme.csv',
 'zips.csv']

In [3]:
Login = pd.read_csv('LoginData.csv')
Login.rename(columns={ Login.columns[0]: "ENTITY"}, inplace=True)

Partner = pd.read_csv('PartnerData.csv')
Partner.rename(columns={ Partner.columns[0]: "ENTITY"}, inplace=True)

Events = pd.read_csv('EventData.csv')
Events.rename(columns={ Events.columns[0]: "ENTITY"}, inplace=True)

SF = pd.read_csv('Salesforce.csv')
SF.rename(columns={ SF.columns[0]: "ENTITY"}, inplace=True)
SF.drop_duplicates(['ENTITY'], inplace=True) #because multiple case records are logged for each user, thereby duplicating users

Zips = pd.read_csv('Zips.csv') #zip code data


In [4]:
Login.loc[:, 'DATE_OF_BIRTH'].head()

0     7/22/1971
1     7/22/1971
2    11/25/1977
3      6/1/1958
4     8/23/1974
Name: DATE_OF_BIRTH, dtype: object

In [5]:
now=pd.Timestamp(DT.datetime.now()) #for age calculation
Login.loc[:, 'DATE_OF_BIRTH'] = pd.to_datetime(Login.loc[:, 'DATE_OF_BIRTH'])
Login.loc[:, 'AGE'] = (now - Login.loc[:, 'DATE_OF_BIRTH']).astype('<m8[Y]')
Login.head()
Login.drop(['DATE_OF_BIRTH'], axis=1, inplace=True)

In [6]:
df = pd.merge(Login, 
              pd.merge(Events, 
                       Partner, 
                       on='ENTITY', how='outer'),
                  on='ENTITY', how='outer')

df.drop_duplicates(['ENTITY'], inplace=True) #occasional duplicate users here, too. I don't need them. 

df['CONTACT'] = np.where(df['ENTITY'].isin(SF['ENTITY']), True,np.nan) #true/false if member actually contacted. 

In [7]:
df.head()
df.set_index('ENTITY', inplace=True) #let's set our index, too:

In [8]:
print(df.isnull().sum()) #checking range of missing values.

FIRST_APP_LOGIN_DATE    276182
FIRST_WEB_LOGIN_DATE     20112
earned_status_adj        20157
hra_completion_date     229520
vc_completion_date      381478
COVERAGE_EFF_DATE        20047
ZIP_CD                   44178
AGE                      20047
eventdesc               498705
partner_name            288187
CONTACT                 457902
dtype: int64


In [10]:
df.head()

,FIRST_APP_LOGIN_DATE,FIRST_WEB_LOGIN_DATE,earned_status_adj,hra_completion_date,vc_completion_date,COVERAGE_EFF_DATE,ZIP_CD,AGE,eventdesc,partner_name,CONTACT
ENTITY,,,,,,,,,,,
1.400067e+09,NaN,4/8/2015,Bronze,NaN,NaN,2/23/2015,77064,47.0,NaN,NaN,NaN
1.400078e+09,NaN,11/21/2016,Bronze,NaN,NaN,1/1/2014,97405,40.0,NaN,NaN,1.0
1.400081e+09,4/3/2017,2/8/2013,Platinum,1/1/2018 0:00,3/9/2018 0:00,1/1/2013,32832,60.0,NaN,APPLE,NaN
1.400100e+09,NaN,1/11/2013,Silver,1/29/2018 0:00,2/13/2018 0:00,1/1/2011,55442,44.0,NaN,NaN,1.0
1.400104e+09,4/12/2016,4/12/2016,Platinum,12/1/2017 0:00,NaN,11/3/2014,61115,53.0,NaN,Google Fit,1.0


In [11]:
#need to bin ages

df['AGE'] = pd.cut(df['AGE'], [0, 18, 34, 49, 64, 80], labels=['0-18', '19-34', '35-49', '50-64', '65-80'])

df['AGE'].tail()

ENTITY
1.418832e+09    NaN
1.422994e+09    NaN
1.424272e+09    NaN
1.406092e+09    NaN
1.424398e+09    NaN
Name: AGE, dtype: category
Categories (5, object): [0-18 < 19-34 < 35-49 < 50-64 < 65-80]

In [12]:
df.loc[:,df.columns != 'AGE' ].head()

,FIRST_APP_LOGIN_DATE,FIRST_WEB_LOGIN_DATE,earned_status_adj,hra_completion_date,vc_completion_date,COVERAGE_EFF_DATE,ZIP_CD,eventdesc,partner_name,CONTACT
ENTITY,,,,,,,,,,
1.400067e+09,NaN,4/8/2015,Bronze,NaN,NaN,2/23/2015,77064,NaN,NaN,NaN
1.400078e+09,NaN,11/21/2016,Bronze,NaN,NaN,1/1/2014,97405,NaN,NaN,1.0
1.400081e+09,4/3/2017,2/8/2013,Platinum,1/1/2018 0:00,3/9/2018 0:00,1/1/2013,32832,NaN,APPLE,NaN
1.400100e+09,NaN,1/11/2013,Silver,1/29/2018 0:00,2/13/2018 0:00,1/1/2011,55442,NaN,NaN,1.0
1.400104e+09,4/12/2016,4/12/2016,Platinum,12/1/2017 0:00,NaN,11/3/2014,61115,NaN,Google Fit,1.0


We must interpret missing values before we can address them. A missing value in the login columns isn't missing data per se; rather, it (normally) indicates that the user hasn't used the specified platform. In my experience, it is safer to treat these as FALSEs or NULLs rather than missing values. As such, I will convert these sorts of columns into booleans.   

In [13]:
mydata = df.loc[:, (df.columns != 'AGE') & (df.columns != 'earned_status_adj')].notnull().astype('int')
mydata = pd.merge(mydata,df[['AGE','earned_status_adj']],left_index=True, right_index=True)

In [14]:
mydata = pd.get_dummies(df, columns=['earned_status_adj'], drop_first=False)


In [15]:
mydata.loc[:, 'AGE'] = mydata.loc[:, 'AGE'].dropna()

In [16]:
#let's make some dummies!

mydata = pd.get_dummies(mydata, columns=['AGE'], drop_first=True)

In [17]:
#I also want to get a sort of "waterfall" count...
#Basically, this is the count of conv ersions for each feature

(mydata == 1).sum(axis=1)#sum of conversions by row (member)
(mydata == 1).sum(axis=0)#sum of conversions by column (feature)
 

FIRST_APP_LOGIN_DATE               0
FIRST_WEB_LOGIN_DATE               0
hra_completion_date                0
vc_completion_date                 0
COVERAGE_EFF_DATE                  0
ZIP_CD                             0
eventdesc                          0
partner_name                       0
CONTACT                        78206
earned_status_adj_Bronze      251615
earned_status_adj_Gold         83230
earned_status_adj_Platinum     68237
earned_status_adj_Silver      112869
AGE_19-34                     131686
AGE_35-49                     205959
AGE_50-64                     158120
AGE_65-80                      14843
dtype: int64

In [ ]:
#let's then order our columns appropriately
mydata = mydata[mydata.sum().sort_values(ascending = False).index]
mydata.head()

In [ ]:
#finally, our waterfall
ax = (mydata == 1).sum(axis=0).plot.bar() 

We see roughly equivalent eff dates, login dates, and earned statuses, which is not surprising because these are almost defaults. Web login requires some member action, but it's not until we see HRA through eventdesc that we have some feature columns requiring user agency. I therefore think it makes sense to drop coverage eff date and earned status. 

In [ ]:
mydata.drop(['COVERAGE_EFF_DATE'], axis=1, inplace=True)

In [ ]:
ax = (mydata == 1).sum(axis=0).plot.bar() #much better!

In [ ]:
print(mydata.mean(axis=0))

In [ ]:
mydata.groupby(mydata.sum(axis=1)).agg(sum) #users grouped by count of feature conversions, up to 7 (and apparently not less than 1)

In [ ]:
ax = mydata.groupby(mydata.sum(axis=1)).agg(sum).plot.bar(figsize=(8,8))

It certainly makes sense for the clustered bars to flatten as we move from left to right. It's interesting that four variables applies to more users than two or three variables, but it makes sense for those easy behaviors to be correlated (whereas vc_completion is harder to achieve). The cohort of users who've contacted us seems quite small, but it's important to note that I am using a pretty small set of data for this initial EDA. 

In [ ]:
#Let's see some color!
#number of contact conversions for each grouping
ax=mydata[mydata.loc[:, 'CONTACT']==1].groupby('CONTACT').agg(sum).plot.bar(figsize=(8,6))

In [ ]:
corr = mydata.corr()
f, ax = plt.subplots(figsize=(10, 8))
g = sns.heatmap(corr, mask=np.zeros_like(corr, dtype=np.bool), cmap=sns.diverging_palette(220, 10, as_cmap=True),
            square=True, ax=ax)

#no single variable seems to correlate too strongly to a contact, but partnername is a promising lead

In [ ]:
from sklearn.linear_model import LinearRegression
linreg = LinearRegression()

feature_cols = [i for i in mydata if ('partner' in i)]
X = mydata.loc[:, feature_cols]
y = mydata.loc[:, 'CONTACT']

linreg.fit(X, y)

In [ ]:
y_pred = linreg.predict(X)
mydata.loc[:, 'y_pred'] = y_pred

In [ ]:
print(linreg.intercept_)
print(linreg.coef_)

In [ ]:
#linreg isn't too helpful. Let's try logit. 

mydata.drop(['y_pred'], axis=1, inplace=True)
from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression()


feature_cols = [i for i in mydata if ('CONTACT' not in i and 'pred' not in i)]
X = mydata.loc[:, feature_cols]
y = mydata.loc[:, 'CONTACT']

logreg.fit(X,y)
pred = logreg.predict(X)

In [ ]:
feature_cols


In [ ]:
logreg.predict_proba(X)[0:10]

In [ ]:
mydata.loc[:, 'CONTACT_PROB'] = logreg.predict_proba(X)[:, 1]

In [ ]:
mydata.loc[:, 'CONTACT_PROB'].value_counts().sort_values(ascending=False)

In [ ]:
print(len(mydata[mydata.loc[:, 'CONTACT_PROB'] > 0.6]))
mydata[mydata.loc[:, 'CONTACT_PROB'] < 0.6
      ].head()
#presumably the likeliest contacts. 
#In other words, people for whom all conversions are true have a slightly higher chance of contacting us than 
#people for whom any other sets of conversions are true. That makes sense--but is it helpful?


This, clearly, still is not very helpful. Let's try a non-parametric approach.

In [ ]:
from sklearn.tree import DecisionTreeClassifier

X = mydata.loc[:, feature_cols]
y = mydata.loc[:, 'CONTACT']

treeclf = DecisionTreeClassifier(max_depth=3, random_state=1)
treeclf.fit(X, y)

In [ ]:
pd.DataFrame({'feature':feature_cols, 'importance':treeclf.feature_importances_})

Some comments for next time: 
I wonder if engagement factors will suffice. I am having some difficulty gathering full demographic date (age, gender, etc.), but will try to blend those factors into the final product. 
I also have not fully reckoned with a really severe limitation to this data: we don't know whether these engagement factors were true before the user actually contacted us. It could be that they are more engaged post-conversion. One might with some justice assume that this cohort was more inclined to engage anyway, but that's still an unchastened assumption. 
Finally, 46% contact probability with logistic regression is still less than my goal of >50%. 